# Homework 5 - Part 2
## MET CS 688
### Maryam Asghari

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Installing Transformers


In [ ]:
pip install transformers

## load the pre-trained BERT model


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


We have the main BERT model, a dropout layer to prevent overfitting, and finally a dense layer for classification task

## Imprts

In [1]:
import tensorflow as tf
import pandas as pd

## Large Movie Review Dataset

This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well. Raw text and already processed bag of words formats are provided. See the README file contained in the release for more details.

## Get the Data from the Stanford Repo

italicized text@InProceedings{maas-EtAl:2011:ACL-HLT2011,
  author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
  title     = {Learning Word Vectors for Sentiment Analysis},
  booktitle = {Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies},
  month     = {June},
  year      = {2011},
  address   = {Portland, Oregon, USA},
  publisher = {Association for Computational Linguistics},
  pages     = {142--150},
  url       = {http://www.aclweb.org/anthology/P11-1015}
}

In [ ]:
URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz", 
                                  origin=URL,
                                  untar=True,
                                  cache_dir='.',
                                  cache_subdir='')

84131840/84125825 [==============================] - 4s 0us/step


### Preprossesing and pereparing the data for Bert model

@Credit:
Orhan G. Yalçın
Nov 28, 2020

In [ ]:
# The shutil module offers a number of high-level 
# operations on files and collections of files.
import os
import shutil
# Create main directory path ("/aclImdb")
main_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
# Create sub directory path ("/aclImdb/train")
train_dir = os.path.join(main_dir, 'train')
# Remove unsup folder since this is a supervised learning task
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)
# View the final train folder
print(os.listdir(train_dir))

['neg', 'urls_neg.txt', 'labeledBow.feat', 'urls_unsup.txt', 'urls_pos.txt', 'pos', 'unsupBow.feat']


## Train and Test Split

In [ ]:
# We create a training dataset and a validation 
# dataset from our "aclImdb/train" directory with a 80/20 split.
train = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=30000, validation_split=0.2, 
    subset='training', seed=123)
test = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=30000, validation_split=0.2, 
    subset='validation', seed=123)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
for i in train.take(1):
  train_feat = i[0].numpy()
  train_lab = i[1].numpy()

train = pd.DataFrame([train_feat, train_lab]).T
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
train['DATA_COLUMN'] = train['DATA_COLUMN'].str.decode("utf-8")
train.head()

,DATA_COLUMN,LABEL_COLUMN
0,Canadian director Vincenzo Natali took the art...,1
1,I gave this film 10 not because it is a superb...,1
2,I admit to being somewhat jaded about the movi...,1
3,"For a long time, 'The Menagerie' was my favori...",1
4,A truly frightening film. Feels as if it were ...,0


In [ ]:
for j in test.take(1):
  test_feat = j[0].numpy()
  test_lab = j[1].numpy()

test = pd.DataFrame([test_feat, test_lab]).T
test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
test['DATA_COLUMN'] = test['DATA_COLUMN'].str.decode("utf-8")
test.head()

,DATA_COLUMN,LABEL_COLUMN
0,I can't believe that so much talent can be was...,0
1,This movie blows - let's get that straight rig...,0
2,"The saddest thing about this ""tribute"" is that...",0
3,I'm only rating this film as a 3 out of pity b...,0
4,Something surprised me about this movie - it w...,1


In [ ]:
InputExample(guid=None,
             text_a = "Hello, world",
             text_b = None,
             label = 1)

InputExample(guid=None, text_a='Hello, world', text_b=None, label=1)

In [ ]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


* Adam as our optimizer :
Adam is an optimization algorithm that can be used instead of the classical stochastic gradient descent procedure to update network weights iterative based in training data.

* CategoricalCrossentropy as our loss function:
Categorical crossentropy is a loss function that is used in multi-class classification tasks. These are tasks where an example can only belong to one out of many possible categories, and the model must decide which one.

* SparseCategoricalAccuracy as our accuracy metric :
This metric creates two local variables, `total` and `count` that are used to
compute the frequency with which `y_pred` matches `y_true`. This frequency is
ultimately returned as `sparse categorical accuracy`: an idempotent operation
that simply divides `total` by `count`.

* epochs ( here only 2):
Epoch refers to one cycle through the full training dataset, or the number of times that a learning algorithm is going to update the weights.

In [ ]:

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

CPU times: user 13min 58s, sys: 12min 28s, total: 26min 26s

Wall time: 36min 31s


In [ ]:
## Testing the model to predict sentiment score of a few lines 

In [ ]:
pred_sentences = ['This was an awesome movie. I watch it twice my time watching this beautiful movie if I have known it was this good',
                  'One of the worst movies of all time. I cannot believe I wasted two hours of my life for this movie', 'I loved this movie It was amazing','It wan\'t so bad, it was okay' , 'i really enjoyed this movie', 'Apple watch is a wearable device','very bad movie']

In [ ]:

tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])

This was an awesome movie. I watch it twice my time watching this beautiful movie if I have known it was this good : 
 Positive
One of the worst movies of all time. I cannot believe I wasted two hours of my life for this movie : 
 Negative
I loved this movie It was amazing : 
 Positive
It wan't so bad, it was okay : 
 Positive
i really enjoyed this movie : 
 Positive
Apple watch is a wearable device : 
 Positive
very bad movie : 
 Negative


In [56]:
tf_predictions

<tf.Tensor: shape=(7, 2), dtype=float32, numpy=
array([[1.6577400e-03, 9.9834228e-01],
       [9.9928421e-01, 7.1577996e-04],
       [1.7036176e-03, 9.9829644e-01],
       [3.6320204e-03, 9.9636793e-01],
       [1.0716446e-03, 9.9892837e-01],
       [1.7132899e-01, 8.2867098e-01],
       [9.9922752e-01, 7.7250513e-04]], dtype=float32)>

In [57]:
tf_outputs

TFSequenceClassifierOutput([('logits',
                             <tf.Tensor: shape=(7, 2), dtype=float32, numpy=
                             array([[-3.3077235 ,  3.0929174 ],
                                    [ 3.6101005 , -3.631321  ],
                                    [-3.2740688 ,  3.0992274 ],
                                    [-3.0653615 ,  2.548966  ],
                                    [-3.5101855 ,  3.3273032 ],
                                    [-0.957094  ,  0.61914355],
                                    [ 3.5352743 , -3.6298249 ]], dtype=float32)>)])

In [55]:
tf.argmax(tf_predictions, axis=1)

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([1, 0, 1, 1, 1, 1, 0])>

In [58]:
tf_outputs

TFSequenceClassifierOutput([('logits',
                             <tf.Tensor: shape=(7, 2), dtype=float32, numpy=
                             array([[-3.3077235 ,  3.0929174 ],
                                    [ 3.6101005 , -3.631321  ],
                                    [-3.2740688 ,  3.0992274 ],
                                    [-3.0653615 ,  2.548966  ],
                                    [-3.5101855 ,  3.3273032 ],
                                    [-0.957094  ,  0.61914355],
                                    [ 3.5352743 , -3.6298249 ]], dtype=float32)>)])

### Importing our titles 

In [138]:
data = pd.read_csv('/content/drive/MyDrive/titles50.csv')

In [139]:
data.title

0     Smart Wearables for Sports and Fitness Market ...
1     Health and fitness wearables have boomed durin...
2     Wearables shipments surged in 2020, thanks to ...
3       IDC: Wearables Market Grew 28.4 Percent in 2020
4     Researchers discover new approach to thermal p...
5     Future Amazfit wearables could have ECG and bl...
6     Apple continues to dominate wearables market w...
7     4 Stocks to Watch Amid Booming Demand for Wear...
8     Style-deprived fitness bands have helped cheap...
9     IDC: Pandemic shifted consumer spending to wea...
10    The Zacks Analyst Blog Highlights: Apple, Goog...
11    Mother's day: Indulge your mom with the most p...
12    Huawei Talkband B6 review: a fitness tracker a...
13    Honor could soon launch its cheapest fitness t...
14    Fitness Trackers And Smartwatches That Play Mu...
15    Fitness Tracker Market to Gain Impetus; High D...
16    Peloton, Oura, and Whoop: High-performance app...
17                             Wearables up 28% 

### Use Bert model to predict the score of these titles 

In [140]:
for text in data.title:
  tf_batch = tokenizer(text, max_length=128, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  labels = ['Negative','Positive']
  label = tf.argmax(tf_predictions, axis=1)
  #label = label.numpy()
  #for i in range(len(pred_sentences)):
  score = tf_outputs.logits[0][1]
  print(score)

tf.Tensor(1.9581282, shape=(), dtype=float32)
tf.Tensor(1.6802732, shape=(), dtype=float32)
tf.Tensor(1.982404, shape=(), dtype=float32)
tf.Tensor(1.9707432, shape=(), dtype=float32)
tf.Tensor(2.2224212, shape=(), dtype=float32)
tf.Tensor(0.8580765, shape=(), dtype=float32)
tf.Tensor(1.4775922, shape=(), dtype=float32)
tf.Tensor(1.1031213, shape=(), dtype=float32)
tf.Tensor(-1.7688468, shape=(), dtype=float32)
tf.Tensor(0.60215217, shape=(), dtype=float32)
tf.Tensor(2.1539264, shape=(), dtype=float32)
tf.Tensor(0.57736, shape=(), dtype=float32)
tf.Tensor(1.0083851, shape=(), dtype=float32)
tf.Tensor(-3.2656395, shape=(), dtype=float32)
tf.Tensor(3.1535764, shape=(), dtype=float32)
tf.Tensor(2.0888603, shape=(), dtype=float32)
tf.Tensor(2.3142123, shape=(), dtype=float32)
tf.Tensor(2.296768, shape=(), dtype=float32)
tf.Tensor(2.6720665, shape=(), dtype=float32)
tf.Tensor(0.9269859, shape=(), dtype=float32)
tf.Tensor(0.97238576, shape=(), dtype=float32)
tf.Tensor(1.89726, shape=(), dtype

### creat a function to get the score and  rate from models output

In [141]:

def bert_Sent(text):
  tf_batch = tokenizer(text, max_length=128, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  #label = tf.argmax(tf_predictions, axis=1)
  #label = label.numpy()
  score = tf_outputs.logits[0][1]
  return score

In [142]:
import re

In [143]:
def get_score(text):
  score   = re.search('tf.Tensor\((.+?), shape',text).group(1)
  return score 

In [ ]:

def bert_Sent(text):
  tf_batch = tokenizer(text, max_length=128, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  #label = tf.argmax(tf_predictions, axis=1)
  #label = label.numpy()
  score = tf_outputs.logits[0][1]
  return score

import re

def get_score(text):
  score   = re.search('tf.Tensor\((.+?), shape',text).group(1)
  return score 

def get_rate(text):
  tf_batch = tokenizer(text, max_length=100, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  labels = ['Negative','Positive']
  label = tf.argmax(tf_predictions, axis=1)
  label = label.numpy()
  return labels[label[0]]

In [4]:
transf_sc  = [bert_Sent(text) for text in data.title]
data['Bert_sentiment'] = transf_sc
data['Bert_sentiment'] = data['Bert_sentiment'].astype(str)
data['Bert_sentiment'] = [float(get_score(text)) for text in data.Bert_sentiment]
tf_rate  = [get_rate(text) for text in data.title]

data['Bert_rate'] = tf_rate 
data.columns

data[['nltk_sentiment','TextBlob_sentiment','Bert_sentiment']] = data[['nltk_sentiment','TextBlob_sentiment', 'Bert_sentiment']].apply(lambda x: pd.Series.round(x,2)) 


data[[ 'afinn_sentimen','nltk_sentiment','TextBlob_sentiment','transformers_score','transformers_label','Bert_sentiment','Bert_rate']]

,afinn_sentimen,nltk_sentiment,TextBlob_sentiment,transformers_score,transformers_label,Bert_sentiment,Bert_rate
0,6,0.85,0.21,1.00,POSITIVE,1.96,Positive
1,1,0.27,0.00,0.97,POSITIVE,1.68,Positive
2,2,0.44,0.20,0.99,POSITIVE,1.98,Positive
3,0,0.00,0.00,0.79,POSITIVE,1.97,Positive
4,0,0.00,0.14,0.99,POSITIVE,2.22,Positive
5,-1,-0.30,0.00,1.00,NEGATIVE,0.86,Positive
6,0,-0.13,0.14,0.99,NEGATIVE,1.48,Positive
7,-1,-0.13,0.00,0.91,POSITIVE,1.10,Positive
8,-2,-0.30,-0.15,1.00,NEGATIVE,-1.77,Negative
9,0,0.00,0.00,0.99,NEGATIVE,0.60,Positive


In [146]:
data.to_excel('/content/drive/MyDrive/titles_sentiments.xlsx', index = False)


In [2]:
data = pd.read_excel("titles_sentiments_theme.xlsx")

In [5]:
data.head(20)

,title,afinn_sentimen,nltk_sentiment,TextBlob_sentiment,transformers_score,transformers_label,Bert_sentiment,Bert_rate,Theme
0,Smart Wearables for Sports and Fitness Market ...,6,0.85,0.21,1.00,POSITIVE,1.96,Positive,"Activity Tracker, Smart Watches, Future market"
1,Health and fitness wearables have boomed durin...,1,0.27,0.00,0.97,POSITIVE,1.68,Positive,"Activity Tracker,Health, lifestyle, Quarantine..."
2,"Wearables shipments surged in 2020, thanks to ...",2,0.44,0.20,0.99,POSITIVE,1.98,Positive,"Activity Tracker, pandemic - Market of Activit..."
3,IDC: Wearables Market Grew 28.4 Percent in 2020,0,0.00,0.00,0.79,POSITIVE,1.97,Positive,"Market of Activity Trackers, Activity Tracker"
4,Researchers discover new approach to thermal p...,0,0.00,0.14,0.99,POSITIVE,2.22,Positive,"wearables , Wearable Technology,"
5,Future Amazfit wearables could have ECG and bl...,-1,-0.30,0.00,1.00,NEGATIVE,0.86,Positive,"wearables , Wearable Technology, wearables fea..."
6,Apple continues to dominate wearables market w...,0,-0.13,0.14,0.99,NEGATIVE,1.48,Positive,"wearables , wearables market, smart watches"
7,4 Stocks to Watch Amid Booming Demand for Wear...,-1,-0.13,0.00,0.91,POSITIVE,1.10,Positive,"wearables,wearables market, stock market"
8,Style-deprived fitness bands have helped cheap...,-2,-0.30,-0.15,1.00,NEGATIVE,-1.77,Negative,"wearables , smart watches,fittness trackers"
9,IDC: Pandemic shifted consumer spending to wea...,0,0.00,0.00,0.99,NEGATIVE,0.60,Positive,"wearables , smart watches, pandemic,wearables ..."
